In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# List of years to scrape
years = range(2019, 2024)

# Initialize an empty list to store dataframes
dfs = []

for year in years:
    # URL of the website
    url = f"https://www.chicagobooth.edu/employmentreport/industry/full-time-{year}"

    # Send a GET request to the URL
    response = requests.get(url)

    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table with id "myTable"
    table = soup.find('table', id='myTable')

    # Check if the table is found
    if table:
        # Use pandas to read the HTML table
        df = pd.read_html(str(table))[0]
        
        # Remove numbers, commas, and trailing spaces from column header names
        df.columns = df.columns.str.replace(r'\d+', '', regex=True).str.replace(',', '').str.strip()
        
        # Keep only rows with salary data
        #df = df.dropna(subset=['Median Salary'])
        
        # Add year column
        df['Year'] = year
        
        # Append to the list of dataframes
        dfs.append(df)
    else:
        print(f"Table with id 'myTable' not found on the page for year {year}.")

# Combine all dataframes
combined_df = pd.concat(dfs, ignore_index=True)


In [10]:
# Convert 'Percent of Hires' to decimal form
combined_df['Percent of Hires'] = combined_df['Percent of Hires'].str.rstrip('%').astype(float) / 100

# Replace "Insufficient data1" with N/A values
combined_df = combined_df.replace("Insufficient data1", pd.NA)

# Drop rows where 'Percent of Hires' is less than 5%
combined_df = combined_df[combined_df['Percent of Hires'] >= 0.05]
print(f"Rows remaining after dropping rows with 'Percent of Hires' < 5%: {len(combined_df)}")

# Reset the index after dropping rows
combined_df = combined_df.reset_index(drop=True)
print(f"Rows remaining after resetting index: {len(combined_df)}")

# Convert 'Year' column to datetime
combined_df['Year'] = pd.to_datetime(combined_df['Year'], format='%Y')
print(f"Rows remaining after converting 'Year' to datetime: {len(combined_df)}")

# Function to convert salary strings to numeric values
def convert_salary(salary):
    if isinstance(salary, str):
        return float(salary.replace('$', '').replace(',', ''))
    return salary

# Apply the conversion function to salary columns
salary_columns = ['Minimum Salary', 'Maximum Salary', 'Median Salary', 'Median Sign-On']
for col in salary_columns:
    combined_df[col] = combined_df[col].apply(convert_salary)
    print(f"Rows remaining after converting '{col}' to numeric: {len(combined_df)}")

# Remove numbers and trailing spaces from the Industry column
combined_df['Industry'] = combined_df['Industry'].str.replace(r'\d+', '', regex=True).str.strip()
print(f"Rows remaining after cleaning 'Industry' column: {len(combined_df)}")

# Rename 'eCommerce and Internet' to 'Ecommerce & Internet'
combined_df['Industry'] = combined_df['Industry'].replace('eCommerce and Internet', 'Ecommerce & Internet')
combined_df['Industry'] = combined_df['Industry'].replace('Ecommerce and Internet', 'Ecommerce & Internet')

print(f"Rows after renaming 'eCommerce and Internet': {len(combined_df)}")



Rows remaining after dropping rows with 'Percent of Hires' < 5%: 46
Rows remaining after resetting index: 46
Rows remaining after converting 'Year' to datetime: 46
Rows remaining after converting 'Minimum Salary' to numeric: 46
Rows remaining after converting 'Maximum Salary' to numeric: 46
Rows remaining after converting 'Median Salary' to numeric: 46
Rows remaining after converting 'Median Sign-On' to numeric: 46
Rows remaining after cleaning 'Industry' column: 46
Rows after renaming 'eCommerce and Internet': 46


In [11]:
combined_df

,Industry,Percent of Hires,Number of Hires,Minimum Salary,Maximum Salary,Median Salary,Median Sign-On,Year
0,Consulting,0.337,163,80000.0,210000.0,158000.0,30000.0,2019-01-01
1,Financial Services,0.313,151,80000.0,250000.0,150000.0,40000.0,2019-01-01
2,Diversified Financial Services,0.072,35,88000.0,180000.0,150000.0,40000.0,2019-01-01
3,Investment Banking/Brokerage,0.104,50,107406.0,175000.0,150000.0,50000.0,2019-01-01
4,Investment Management/Research,0.056,27,100000.0,175000.0,135000.0,35000.0,2019-01-01
5,Private Equity,0.060,29,80000.0,250000.0,150000.0,NaN,2019-01-01
6,Technology,0.207,100,83000.0,168000.0,130000.0,32500.0,2019-01-01
7,Ecommerce & Internet,0.120,58,83000.0,168000.0,130000.0,35000.0,2019-01-01
8,Software,0.050,24,100000.0,160000.0,135000.0,NaN,2019-01-01
9,All,1.000,483,78700.0,250000.0,145000.0,30000.0,2019-01-01


In [12]:
# Save the combined dataframe to a CSV file
csv_filename = '../Data/shapedData.csv'
combined_df.to_csv(csv_filename, index=False)
print(f"Data saved to {csv_filename}")


Data saved to ../Data/shapedData.csv
